In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
import os
from dotenv import load_dotenv
from helpers.llm import initialize_llm, logger, pretty_print_docs
from crewai import Agent, Task, Crew 

import requests

load_dotenv()
llm, _, _ = initialize_llm() 



os.environ["AZURE_API_KEY"] = ''
os.environ["AZURE_API_BASE"] = ''
os.environ["AZURE_API_VERSION"] = '2025-01-01-preview' 



In [ ]:
from crewai_tools import TavilySearchTool, ScrapeWebsiteTool
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# Tool: busca web
search_tool = TavilySearchTool(max_results=5)

scrape_tool = ScrapeWebsiteTool(website_url='https://peps.python.org/')

 

In [ ]:
from crewai_tools import (
    TXTSearchTool,
    PDFSearchTool,
    ScrapeWebsiteTool,
    TavilySearchTool,
)

pdf_tool = PDFSearchTool(
    pdf="data/design-patterns-pt-br.pdf",
    config=dict(
        llm=dict(
            provider="azure_openai",  # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gpt-4",
                deployment_name="chat-deployment",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="azure_openai",  # or openai, ollama, ...
            config=dict(
                model="text-embedding-3-large",
                deployment_name="text-embedding-3-large",
            ),
        ),
        vectordb=dict(
            provider="chroma",
            config=dict(
                dir=f"data/crew-db-design-patterns",
                collection_name="design-patterns",
            ),
        ),
    ),
)


# context = pdf_tool.run("Quais os padrões abordados?")
# print(context)

In [ ]:
pdf_refactory_tool = PDFSearchTool(
    pdf="data/refactoring-en-java.pdf",
    config=dict(
        llm=dict(
            provider="azure_openai",  # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gpt-4",
                deployment_name="chat-deployment",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="azure_openai",  # or openai, ollama, ...
            config=dict(
                model="text-embedding-3-large",
                deployment_name="text-embedding-3-large",
            ),
        ),
        vectordb=dict(
            provider="chroma",
            config=dict(
                dir=f"data/crew-db-refactoring",
                collection_name="refactoring",
            ),
        ),
    ),
)

In [ ]:
# Agente 1: Revisor
revisor = Agent(
    role="Code Reviewer",
    goal="Analisar criticamente trechos de código e sugerir melhorias",
    backstory="Você é um revisor sênior com foco em boas práticas, performance e segurança.",
    tools=[pdf_tool, search_tool],  # ← ferramenta integrada
    llm="azure/chat-deployment",
    provider="azure_openai",
    verbose=True,
)

# Agente 2: Desenvolvedor
desenvolvedor = Agent(
    role="Software Engineer",
    goal="Corrigir e melhorar o código conforme sugestões recebidas",
    backstory="Você é um desenvolvedor experiente e segue à arquitetura de software, boas práticas, performance e segurança, utilizando os principios SOLID. Use as informações do PDF e as regras do PEP8 de refatoração para melhorar o código.",
    tools=[pdf_refactory_tool],  # ← ferramenta integrada
    llm="azure/chat-deployment",
    provider="azure_openai",
    verbose=True,
)

In [ ]:
# Trecho de código original
codigo_original = """
def get_data(items):
    results = []
    for i in range(len(items)):
        results.append(items[i] * 2)
    return results
"""

# codigo_original = """
# public class UserService {
#     public void doStuff(String userName, int age) {
#         if (userName != null && !userName.equals("")) {
#             System.out.println("Nome: " + userName);
#         } else {
#             System.out.println("Nome inválido");
#         }

#         if (age > 0) {
#             System.out.println("Idade: " + age);
#         } else {
#             System.out.println("Idade inválida");
#         }

#         try {
#             Thread.sleep(2000); // simula processo
#         } catch (Exception e) {
#             e.printStackTrace();
#         }
#     }
# }
# """
code_language="Python"
# Task 1: Revisor faz análise
task_revisar = Task(
    description=f"""
Você recebeu o seguinte trecho de código {code_language} para revisar:

```{code_language}
{codigo_original}
Faça uma análise crítica usando sua base de conhecimento e, se necessário, consulte a internet para buscar:

Boas práticas para manipulação de listas em Python utilizando a PEP8

Padrões recomendados para funções puras

Otimizações possíveis

Lembre-se de considerar a tipagem dos objetos de entrada e saída, e se o código é seguro para execução em produção.

Utilizando os principios SOLID e melhores design patterns.

Importante este código deve funcionar corretamente após as melhorias.
""",
    expected_output="Comentários detalhados com sugestões específicas de melhoria, para orientar o desenvolvedor.",
    agent=revisor,
)

# Task 2: Desenvolvedor faz correções
task_corrigir = Task(
    description="Implemente as melhorias sugeridas pelo revisor no código original.",
    expected_output="Código corrigido com explicação breve das mudanças, utilizando os melhores design patterns e refactoring. Retorne o código corrigido.",
    agent=desenvolvedor,
    context=[task_revisar],
)

# Task 3: Revisor revisa novamente
task_revisar_novamente = Task(
    description="Revise novamente o código agora com as correções implementadas. Verifique se está adequado.",
    expected_output="Aprovação final, nesta revisão você pode realizar as melhorias se for necessário. Retorne o código desenvolvedor, e código revisado.",
    agent=revisor,
    context=[task_corrigir],
)

In [ ]:
# Criando a equipe
from crewai import Crew, Process
crew = Crew(
    agents=[revisor, desenvolvedor],
    tasks=[task_revisar, task_corrigir, task_revisar_novamente],
    llm=llm,
    verbose=True,
    process=Process.sequential 
)

resultado = crew.kickoff()
print("\n✅ Resultado final:\n", resultado)

In [ ]:
def multiply_by_two(items):
    """
    Multiply each item in the input list by 2.
    
    Args:
        items (list): A list of numbers to be multiplied by 2.
    
    Returns:
    list: A new list with each item from the input list multiplied by 2.
    """
    return [item * 2 for item in items]

In [ ]:
def get_data(items):
    results = []
    for i in range(len(items)):
        results.append(items[i] * 2)
    return results


items = [1, 2, 3, 4, 5]
results = get_data(items)
print("Resultados:", results)
results = multiply_by_two(items)
print("Resultados:", results)